# 2-推荐系统

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io as sio

## 加载数据

矩阵Y（num_movies×num_user矩阵）存储评级y（i, j）（从1到5）。  
矩阵R是二进制值指示矩阵，其中如果用户j给予电影i评级则R（i,j）= 1，否则R（i,j）= 0

In [2]:
raw_data = sio.loadmat('data/ex8_movies.mat')
raw_data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Thu Dec  1 17:19:26 2011',
 '__version__': '1.0',
 '__globals__': [],
 'Y': array([[5, 4, 0, ..., 5, 0, 0],
        [3, 0, 0, ..., 0, 0, 5],
        [4, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'R': array([[1, 1, 0, ..., 1, 0, 0],
        [1, 0, 0, ..., 0, 0, 1],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}

In [3]:
Y = raw_data['Y']
R = raw_data['R']
Y.shape,R.shape

((1682, 943), (1682, 943))

In [4]:
#把x and theta 压缩为一个向量
def serialize(x,theta):
    return np.concatenate((x.ravel(),theta.ravel()))

In [5]:
#向量转数组 返回2个值 x and theta
def deserialize(param,n_movie,n_user,n_features):
    return param[:n_movie * n_features].reshape(n_movie,n_features),\
           param[n_movie * n_features:].reshape(n_user,n_features)

## 代价函数
<img style="float: left;" src="images/cost.png">

用结果点乘R矩阵就能实现r(i,j)=1的计算

In [6]:
def cost(param,Y,R,n_features):
    n_movies,n_users = Y.shape
    x,theta = deserialize(param,n_movies,n_users,n_features)
    
    delta = x @ theta.T - Y
    inner = np.multiply(delta,R)
    cost = (inner ** 2).sum() / 2
    return cost

In [7]:
#加载数据进行测试是否正确
para_data = sio.loadmat('data/ex8_movieParams.mat')
para_data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Thu Dec  1 17:16:53 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 1.0486855 , -0.40023196,  1.19411945, ...,  0.861721  ,
         -0.69728994,  0.28874563],
        [ 0.78085123, -0.38562591,  0.52119779, ...,  0.70402073,
         -0.48583521, -0.56462407],
        [ 0.64150886, -0.54785385, -0.08379638, ...,  0.83854643,
         -0.69483208, -1.13479631],
        ...,
        [ 0.21952237, -0.20047886,  0.09257965, ...,  0.14595183,
         -0.0431316 ,  0.17830451],
        [ 0.16044028, -0.16015395,  0.23570946, ...,  0.2073381 ,
         -0.33234766,  0.0428813 ],
        [ 0.07677118, -0.19720738,  0.08186445, ...,  0.40012757,
         -0.28747906, -0.03233498]]),
 'Theta': array([[ 0.28544362, -1.68426509,  0.26293877, ...,  0.76723235,
         -1.10460164, -0.25186708],
        [ 0.50501321, -0.45464846,  0.31746244, ...,  1.09306336,
         -1.20029436, -0.39161676],
        [-0.4319

In [8]:
para_x = para_data['X']
para_theta = para_data['Theta']
para_x.shape,para_theta.shape

((1682, 10), (943, 10))

In [9]:
users = 4
movies = 5
features = 3

x_sub = para_x[:movies, :features]
theta_sub = para_theta[:users, :features]
Y_sub = Y[:movies, :users]
R_sub = R[:movies, :users]
param_sub = serialize(x_sub, theta_sub)
#cost符合预期输出22.22
cost(param_sub, Y_sub, R_sub, features)

22.224603725685675

In [10]:
#total cost
param = serialize(para_x,para_theta)
cost(param,Y,R,10)

27918.64012454421

## 梯度函数  
<img style="float: left;" src="images/gradient.png">

In [11]:
def gradient(param,Y,R,n_features):
    n_movies,n_users = Y.shape
    # x(1682, 10),theta(943, 10)
    x,theta = deserialize(param,n_movies,n_users,n_features)
    
    #Y(1682,943)
    delta = x @ theta.T - Y
    r_delta = np.multiply(delta,R)
    # (1682, 10) = (1682,943) @ (943, 10)
    x_grad = r_delta @ theta
    # (943, 10) = (1682,943).T @ (1682, 10)
    theta_grad = r_delta.T @ x
    
    return serialize(x_grad,theta_grad)

In [12]:
gradient(param,Y,R,10)

array([-6.26184144,  2.45936046, -6.87560329, ..., -6.56073746,
        5.20459188,  2.65003952])

## 正则化代价函数
<img style="float: left;" src="images/reg_cost.png">

In [13]:
def reg_cost(param,Y,R,n_features,reg):
    n_movies,n_users = Y.shape
    x,theta = deserialize(param,n_movies,n_users,n_features)
    
    theta_term = (theta ** 2).sum() * reg / 2
    x_term = (x ** 2).sum() * reg / 2
    return cost(param,Y,R,n_features) + theta_term + x_term

更便捷的写法  
后面的正则化就是全参数开方相加
```python
def regularized_cost(param, Y, R, n_features, reg=1):
    reg_term = np.power(param, 2).sum() * (reg / 2)

    return cost(param, Y, R, n_features) + reg_term
```

In [14]:
#预期输出31.34
reg_cost(param_sub, Y_sub, R_sub, features, reg=1.5)

31.34405624427422

In [15]:
reg_cost(param, Y, R, 10, reg=1)  # total regularized cost

32520.682450229557

## 正则化梯度函数
\begin{aligned} \frac{\partial J}{\partial x_{k}^{(i)}} &=\sum_{j : r(i, j)=1}\left(\left(\theta^{(j)}\right)^{T} x^{(i)}-y^{(i, j)}\right) \theta_{k}^{(j)}+\lambda x_{k}^{(i)} \\ \frac{\partial J}{\partial \theta_{k}^{(j)}} &=\sum_{i : r(i, j)=1}\left(\left(\theta^{(j)}\right)^{T} x^{(i)}-y^{(i, j)}\right) x_{k}^{(i)}+\lambda \theta_{k}^{(j)} \end{aligned}

In [16]:
def reg_gradient(param,Y,R,n_features,reg):
    n_movies,n_users = Y.shape
     # x(1682, 10),theta(943, 10)
    x,theta = deserialize(param,n_movies,n_users,n_features)
    
    #Y(1682,943)
    delta = x @ theta.T - Y
    r_delta = np.multiply(delta,R)
    x_grad = r_delta @ theta + reg * x
    theta_grad = r_delta.T @ x + reg * theta
    return serialize(x_grad,theta_grad)

一种更便捷的方式  
正则化分别对应作用于每个参数，展开为向量时正好是对应相加
```python
def regularized_gradient(param, Y, R, n_features, reg=1):
    grad = gradient(param, Y, R, n_features)
    reg_term = reg * param

    return grad + reg_term
```

In [17]:
reg_gradient(param_sub, Y_sub, R_sub, features, reg=1.5)

array([ -0.95596339,   6.97535514,  -0.10861109,   0.60308088,
         2.77421145,   0.25839822,   0.12985616,   4.0898522 ,
        -0.89247334,   0.29684395,   1.06300933,   0.66738144,
         0.60252677,   4.90185327,  -0.19747928, -10.13985478,
         2.10136256,  -6.76563628,  -2.29347024,   0.48244098,
        -2.99791422,  -0.64787484,  -0.71820673,   1.27006666,
         1.09289758,  -0.40784086,   0.49026541])

## 模型训练和电影推荐

In [18]:
movie_idx = {}
f = open('data/movie_ids.txt',encoding= 'gbk')
for line in f:
    tokens = line.split(' ')
    tokens[-1] = tokens[-1][:-1]
    movie_idx[int(tokens[0]) - 1] = ' '.join(tokens[1:])

In [19]:
movie_idx[0]

'Toy Story (1995)'

##### 使用练习提供的评分作为自己的喜好

In [20]:
ratings = np.zeros(1682)

ratings[0] = 4
ratings[6] = 3
ratings[11] = 5
ratings[53] = 4
ratings[63] = 5
ratings[65] = 3
ratings[68] = 5
ratings[97] = 2
ratings[182] = 4
ratings[225] = 5
ratings[354] = 5

print('Rated {0} with {1} stars.'.format(movie_idx[0], str(int(ratings[0]))))
print('Rated {0} with {1} stars.'.format(movie_idx[6], str(int(ratings[6]))))
print('Rated {0} with {1} stars.'.format(movie_idx[11], str(int(ratings[11]))))
print('Rated {0} with {1} stars.'.format(movie_idx[53], str(int(ratings[53]))))
print('Rated {0} with {1} stars.'.format(movie_idx[63], str(int(ratings[63]))))
print('Rated {0} with {1} stars.'.format(movie_idx[65], str(int(ratings[65]))))
print('Rated {0} with {1} stars.'.format(movie_idx[68], str(int(ratings[68]))))
print('Rated {0} with {1} stars.'.format(movie_idx[97], str(int(ratings[97]))))
print('Rated {0} with {1} stars.'.format(movie_idx[182], str(int(ratings[182]))))
print('Rated {0} with {1} stars.'.format(movie_idx[225], str(int(ratings[225]))))
print('Rated {0} with {1} stars.'.format(movie_idx[354], str(int(ratings[354]))))

Rated Toy Story (1995) with 4 stars.
Rated Twelve Monkeys (1995) with 3 stars.
Rated Usual Suspects, The (1995) with 5 stars.
Rated Outbreak (1995) with 4 stars.
Rated Shawshank Redemption, The (1994) with 5 stars.
Rated While You Were Sleeping (1995) with 3 stars.
Rated Forrest Gump (1994) with 5 stars.
Rated Silence of the Lambs, The (1991) with 2 stars.
Rated Alien (1979) with 4 stars.
Rated Die Hard 2 (1990) with 5 stars.
Rated Sphere (1998) with 5 stars.


In [21]:
Y = raw_data['Y']
R = raw_data['R']
#插入自己的评价
Y = np.insert(Y, 0, ratings, axis=1)  # now I become user 0
R = np.insert(R, 0, ratings != 0, axis=1)

Y.shape,R.shape

((1682, 944), (1682, 944))

In [22]:
n_features = 10
n_movie, n_user = Y.shape
reg = 10

In [23]:
x = np.random.standard_normal((n_movie, n_features))
theta = np.random.standard_normal((n_user, n_features))

x.shape, theta.shape

((1682, 10), (944, 10))

In [24]:
param = serialize(x, theta)
param.shape

(26260,)

In [25]:
#均值归一化
Y_norm = Y - Y.mean()
Y_norm.mean()

4.6862111343939375e-17

In [26]:
import scipy.optimize as opt
res = opt.minimize(fun=reg_cost,
                   x0=param,
                   args=(Y_norm, R, n_features, reg),
                   method='TNC',
                   jac=reg_gradient)
#这里很慢
res

     fun: 69380.70267951078
     jac: array([ 1.52816693e-05,  5.76434287e-05,  4.49391708e-05, ...,
        2.74691081e-05, -7.34793906e-08, -7.03778045e-06])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 1229
     nit: 53
  status: 1
 success: True
       x: array([-1.03579937,  0.2885878 , -0.51889949, ..., -1.28947283,
       -0.19761547,  0.12587405])

In [27]:
x_trained, theta_trained = deserialize(res.x, n_movie, n_user, n_features)
x_trained.shape, theta_trained.shape

((1682, 10), (944, 10))

In [28]:
prediction = x_trained @ theta_trained.T

In [29]:
#预测时把去掉的均值加回来
my_preds = prediction[:, 0] + Y.mean()

In [30]:
idx = np.argsort(my_preds)[::-1]  # Descending order
idx.shape

(1682,)

In [31]:
print("Top 10 movie predictions:")
for i in range(10):
    j = int(idx[i])
    print('Predicted rating of {0} for movie {1}.'.format(str(float(my_preds[j])), movie_idx[j]))

Top 10 movie predictions:
Predicted rating of 4.2826310452450835 for movie Titanic (1997).
Predicted rating of 4.114989509573208 for movie Star Wars (1977).
Predicted rating of 3.9804480582334 for movie Raiders of the Lost Ark (1981).
Predicted rating of 3.9058053430402118 for movie Good Will Hunting (1997).
Predicted rating of 3.8881765512048103 for movie Shawshank Redemption, The (1994).
Predicted rating of 3.874608392296945 for movie Braveheart (1995).
Predicted rating of 3.8717879145930683 for movie Return of the Jedi (1983).
Predicted rating of 3.865271422196603 for movie Empire Strikes Back, The (1980).
Predicted rating of 3.762376952539388 for movie Terminator 2: Judgment Day (1991).
Predicted rating of 3.7539430105374594 for movie As Good As It Gets (1997).


 和预期结果有些出入，pdf中说的可能是初始化不同的原因，不知道是不是这个原因,应该也和特征个数设置有关